In [1]:
import pandas as pd
import numpy as np

In [2]:

import os, platform
base_path = "data/open/"

def file_exist_check(base_path):
    if os.path.exists(f"{base_path}train_data.csv"):
        print(f"{base_path} 경로에 파일이 이미 있음")
        return
    
    if platform.system() == "Linux":
        print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
        !wget https://bit.ly/dacon-klue-open-zip
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        !unzip dacon-klue-open-zip -d data/klue
    else:
        print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 하고
              실습 경로 {base_path}에 옮겨주세요.""")
    return
    
file_exist_check(base_path) 

data/open/ 경로에 파일이 이미 있음


In [3]:
train = pd.read_csv(f"{base_path}train_data.csv")
test = pd.read_csv(f"{base_path}test_data.csv")
train.shape, test.shape

((45654, 3), (9131, 2))

In [4]:
topic = pd.read_csv(f"{base_path}topic_dict.csv")
topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [5]:
import re

def preprocessing(text):
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub('[\s]+', ' ', text)
    # 영문자를 소문자로 만듭니다.
    text = text.lower()
    return text

In [6]:
train["title"] = train["title"].map(preprocessing)
test["title"] = test["title"].map(preprocessing)

In [7]:
small_text = "버스의 운행시간을 문의합니다. 어?!"
small_text

'버스의 운행시간을 문의합니다. 어?!'

In [8]:
from konlpy.tag import Kkma
from konlpy.tag import Mecab

In [10]:
from konlpy.tag import Okt

sample_text = Okt().pos(small_text, stem=True)
sample_text

[('버스', 'Noun'),
 ('의', 'Josa'),
 ('운행', 'Noun'),
 ('시간', 'Noun'),
 ('을', 'Josa'),
 ('문의', 'Noun'),
 ('하다', 'Verb'),
 ('.', 'Punctuation'),
 ('어', 'Eomi'),
 ('?!', 'Punctuation')]

In [28]:
def okt_clean(text):
    clean_text = []
    pos_text = Okt().pos(text, stem=True)
    for word, tag in pos_text:
        if tag not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word)
            
    return ' '.join(clean_text)

okt_clean("지금 배가 너무 고프다 ㅠㅠ :)")

'지금 배 너무 고프다 ㅠㅠ'

In [31]:
from tqdm import tqdm
tqdm.pandas() 

train['title_text'] = train['title'].progress_apply(okt_clean)
test['title_text'] = test['title'].progress_apply(okt_clean)

100%|██████████| 9131/9131 [00:06<00:00, 1356.89it/s]


In [32]:
train['title_text'].head()

0                인천 핀란드 항공기 결항 휴가 철 여행객 분통
1           실리콘밸리 넘어서다 구글 15조원 들이다 전역 거점 화
2          이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3    nyt 클린턴 측근 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4                시진핑 트럼프 중미 무역 협상 조속 타결 희망
Name: title_text, dtype: object

In [33]:
test['title_text'].head()

0              유튜브 내달 2일 까지 크리에이터 지원 공간 운영
1                 어버이날 맑다 흐려지다 남부 지방 옅다 황사
2                내년 국가 rd 평가 때 논문 건수 반영 않다
3          김명자 신임 과총 회장 원로 젊다 과학자 지혜 모으다 것
4    회색 인간 작가 김 동식 양심 고백 등 새 소설 집 2 권 추다 간
Name: title_text, dtype: object

In [34]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = [ '합니다', '하는', '할', '하고', '한다', 
             '그리고', '입니다', '그', '등', '이런', '및','제', '더']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [37]:
train["title"] = train["title_text"].map(remove_stopwords)
test["title"] = test["title_text"].map(remove_stopwords)
train["title"]

0                    인천 핀란드 항공기 결항 휴가 철 여행객 분통
1               실리콘밸리 넘어서다 구글 15조원 들이다 전역 거점 화
2              이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3        nyt 클린턴 측근 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4                    시진핑 트럼프 중미 무역 협상 조속 타결 희망
                         ...                  
45649             kb 금융 미국 ib 스티펠 제휴 선진국 시장 공략
45650        1 보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토
45651        게시판 키우다 증권 2020 키우다 영웅 전 실전 투자 대회
45652                   답변 하다 배 기동 국립 중앙 박물관 장
45653    2020 한국 인터넷 기자 상 시상식 내달 1일 개최 특별상 김성후
Name: title, Length: 45654, dtype: object

In [36]:
test["title"]

0            유튜브 내달 2일까지 크리에이터 지원 공간 운영
1               어버이날 맑다가 흐려져 남부지방 옅은 황사
2           내년부터 국가rd 평가 때 논문건수는 반영 않는다
3       김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4          회색인간 작가 김동식 양심고백 새 소설집 2권 출간
                     ...               
9126       인천 오후 3시35분 대설주의보 눈 3 1cm 쌓여
9127      노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128       40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129      게시판 아리랑tv 아프리카개발은행 총회 개회식 생중계
9130    유영민 과기장관 강소특구는 지역 혁신의 중심 지원책 강구
Name: title, Length: 9131, dtype: object

In [38]:
X_train_text = train['title_text']
X_test_text = test['title_text']

In [40]:
label_name = 'topic_idx'

In [41]:

y_train = train[label_name]
y_train.value_counts()

4    7629
2    7362
5    6933
6    6751
1    6222
3    5933
0    4824
Name: topic_idx, dtype: int64